# Find a Legend Challenge - Submission Notebook

This notebook is designed to work with as few external files needed.
Unfortunately the OCR Tool use (tesseract) needs to be installed manually as a executable.
For a windows machine we suggest using the installer provided here: https://github.com/UB-Mannheim/tesseract/wiki (standard configuration will save the files to expected the target location). Feel free to install tesseract in other ways, but notice that we cannot assure all paths are working correctly.


Competition:  https://xeek.ai/challenges/extract-crossplot-markers <br>
Repository: https://github.com/REDA-solutions/PlotLegendDetectionCV

In [1]:
team_name = 'REDA solutions' 
model_name = 'legend_detection_model'

## Imports

In [2]:
! pip install -r requirements.txt --user

     ---------------------------------------- 14.7/14.7 MB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 4.8 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.23.1 requires regex!=2019.12.17, which is not installed.
tensorflow 2.7.0 requires h5py>=2.9.0, which is not installed.


In [3]:
import os
import platform
import sys
import numpy as np
import pandas as pd
from glob import glob
import torch
import re

from models_ocr.preprocessing.preprocessor import Preprocessor # import of our class --> see repository
from models_ocr.pytesseract_model import PytesseractModel # import of our class --> see repository

## Description

![Model Architecture](misc/model_architecture.png)

## Submission inference pipeline

In [4]:
from time import perf_counter

In [5]:
TEST_DATA_ROOT = "raw_data/helvetios_challenge_dataset_test"
TEST_IMAGE_DATASET_PATH = f"{TEST_DATA_ROOT}/images"
TEST_LABELS_DATA_PATH = f"{TEST_DATA_ROOT}/labels"
TEST_INFERENCE_RESULTS_PATH = f"results"

In [6]:
def run_inference_pipeline(TEST_IMAGE_DATASET_PATH, TEST_INFERENCE_RESULTS_PATH):
   
   print(f"* OS                          : {platform.system()}, {platform.release()}")
   python_version = str(sys.version).replace('\n', ' ')
   print(f"* Python version              : {python_version}")

   os.makedirs(TEST_INFERENCE_RESULTS_PATH, exist_ok=True)
         
   ts_start = perf_counter()
   
   model = torch.hub.load('../yolov5/', 'custom', path='models_detection/best.pt', source='local')
   model.conf = 0.2 # define minimal confidence for found legends
   preprocessor = Preprocessor(deskew=True)
   tesseract = PytesseractModel(preprocessor=preprocessor, confidence=15, custom_config=r"-l eng --psm 11")

   imgs = TEST_IMAGE_DATASET_PATH + r'/*.png'  
   imgs = list(glob(imgs))
   
   results = []
   sample_names = []

   for img in imgs:
      # apply best.pt to the images in the source path
      legends = model(img)
      # crop the image - the next steps will only look at the part of the img detected as legend (kan be multiple frames)
      legends = legends.crop()
      if len(legends) == 0:
         # action if no legend was found
         results.append(np.nan)
      else:
         predictions = []
         for legend_ in legends:
            legend = legend_['im']
            # use the parameterized pytesseract model to predict the legend text
            prediction = tesseract.predict(legend)
            predictions.extend(prediction)

         # some alignments to assure the detected text matches the expected syntax of legend values
         reg = re.compile('/[^0-9]/g')
         predictions = [s for s in predictions if not any(chr.isdigit() for chr in s)]
         predictions = [s for s in predictions if len(s)!=1 or not s.islower()]
         prediction_str = "["
         for word in predictions: prediction_str += f"'{word}' "
         prediction_str = prediction_str.strip()
         prediction_str += "]"
         if prediction_str == "[]":
            prediction_str = np.nan
         results.append(prediction_str)
      sample_names.append(img.split("\\")[1])
   
   ts_after_test = perf_counter()
   
   print(f"Inference time: {ts_after_test-ts_start:.2f} sec.")
   
   inference_results = {'sample_name': sample_names,
                        'legend': results}
   inference_results_df = pd.DataFrame(inference_results)
      
   inference_results_df.to_csv(f"{TEST_INFERENCE_RESULTS_PATH}/{team_name}_{model_name}_results.csv", index = False)
   
   print(inference_results_df)

   print(f"The submission file   : {TEST_INFERENCE_RESULTS_PATH}/{team_name}_{model_name}_results.csv")


In [7]:
run_inference_pipeline(TEST_IMAGE_DATASET_PATH, TEST_INFERENCE_RESULTS_PATH)

* OS                          : Windows, 10
* Python version              : 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]


YOLOv5  v7.0-10-g10c025d Python-3.9.7 torch-1.13.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
Saved 1 image to runs\detect\exp303
Saved results to runs\detect\exp303

Saved 1 image to runs\detect\exp304
Saved results to runs\detect\exp304

Saved 1 image to runs\detect\exp305
Saved results to runs\detect\exp305

Saved 1 image to runs\detect\exp306
Saved results to runs\detect\exp306

Saved 1 image to runs\detect\exp307
Saved results to runs\detect\exp307

Saved 1 image to runs\detect\exp308
Saved results to runs\detect\exp308

Saved 1 image to runs\detect\exp309
Saved results to runs\detect\exp309

Saved 1 image to runs\detect\exp310
Saved results to runs\detect\exp310

Saved 1 image to runs\detect\exp311
Saved results to runs\detect\exp311

Saved 1 image to runs\detect\exp312
Saved results to runs\detect\exp312

Saved 1 image to runs\detect\exp313
Saved results to runs\detect\exp313

Saved 1 image to runs\dete

Inference time: 85.58 sec.
                  sample_name                                      legend
0    20220915195651573677.png       ['drt' 'qn' '©' 'deo' 'RE' '@' 'ber']
1    20220915195652176684.png                                 ['On' 'Os']
2    20220915195652713648.png                                ['Mobility']
3    20220915195653681519.png                                     ['exe']
4    20220915195654121931.png                                         NaN
..                        ...                                         ...
295  20220915195931225632.png                         ['area' 'ad' 'IDA']
296  20220915195931982187.png                               ['+' 'Canes']
297  20220915195932679590.png                                     ['ge.']
298  20220915195933359877.png         ['MeV' 'Output' '(bar/m)' '&' 'x”']
299  20220915195934085444.png  ['ME' '@' 'power' '@' 'Titanium' '©' 'ax']

[300 rows x 2 columns]
The submission file   : results/REDA solutions_legend_detecti